# Pipeline Scoring Example

This is a demo notebook showing how to use the `orion.analysis.analyze` function on one of the
signals from the Demo Dataset and afterwards evaluate how well the pipeline performed.

In this case, we will be processing the `S-1` signal with the `lstm_dynamic_threshold.json`
pipeline.

## 1. Load the data

In the first step, we setup the environment and load the **S-1** signal from the **Demo Dataset**,

To do so, we need to import the `orion.data.load_signal` function and call it passing
the `'S-1'` name.

In [1]:
import logging;

logging.basicConfig(level=logging.ERROR)
logging.getLogger().setLevel(level=logging.ERROR)

import warnings
warnings.simplefilter("ignore")

In [2]:
from orion.data import load_signal

train = load_signal('S-1-train')
test = load_signal('S-1-test')
train.head()

,timestamp,value
0,1222819200,-0.366359
1,1222840800,-0.394108
2,1222862400,0.403625
3,1222884000,-0.362759
4,1222905600,-0.370746


## 2. Detect anomalies using a pipeline

Once we have the data, let us try to use the LSTM pipeline to analyze it and search for anomalies.

In order to do so, we will have import the `orion.analysis.analyze` function and pass it
the loaded data and the path to the pipeline JSON that we want to use.

The output will be a ``pandas.DataFrame`` containing a table with the detected anomalies.

In [3]:
from orion.analysis import analyze

hyperparameters = {
    'keras.Sequential.LSTMTimeSeriesRegressor#1': {
        'epochs': 5,
        'verbose': True
    }
}

pipeline = 'lstm_dynamic_threshold'

anomalies = analyze(pipeline, train, test, hyperparameters)
anomalies

Using TensorFlow backend.


Train on 2054 samples, validate on 514 samples
Epoch 1/5
2054/2054 [==============================] - 12s 6ms/step - loss: 0.4788 - mse: 0.4788 - val_loss: 0.4413 - val_mse: 0.4413
Epoch 2/5
2054/2054 [==============================] - 10s 5ms/step - loss: 0.4239 - mse: 0.4239 - val_loss: 0.4340 - val_mse: 0.4340
Epoch 3/5
2054/2054 [==============================] - 10s 5ms/step - loss: 0.4267 - mse: 0.4267 - val_loss: 0.4313 - val_mse: 0.4313
Epoch 4/5
2054/2054 [==============================] - 10s 5ms/step - loss: 0.4203 - mse: 0.4203 - val_loss: 0.4298 - val_mse: 0.4298
Epoch 5/5
7081/7081 [==============================] - 9s 1ms/step


,start,end,score
0,1398600000,1406030400,0.434961


## 3. Evaluate performance

In this next step we will load some already known anomalous intervals and evaluate how
good our anomaly detection was by comparing those with our detected intervals.

For this, we will first load the known anomalies for the signal that we are using:

In [4]:
from orion.data import load_anomalies

known_anomalies = load_anomalies('S-1')
known_anomalies

,start,end
0,1398168000,1407823200


Afterwards, we pass the ground truth, the detected anomalies and the original data
to the `orion.metrics.contextual_accuracy` and `orion.metrics.contextual_f1_score` functions in order
to compute a score that indicates how good our anomaly detection was. You can read more about how to evaluate a pipeline [here](https://signals-dev.github.io/Orion/user_guides/evaluation_doc.html).

In [5]:
from orion.evaluation import contextual_accuracy, contextual_f1_score

contextual_accuracy(known_anomalies, anomalies, test)

0.9859481582537517

In [6]:
contextual_f1_score(known_anomalies, anomalies, test)

0.8697850974171117